# Report Basics of Mobile Robotics - 2024

## Table of Contents
0. [Introduction](#Introduction)
1. [Setup](#Partie-1-:-Setup)
    - 1.1 : Physical setup
    - 1.2 : Code setup
2. [Initial tuning](#Partie-2-:-Initial-tuning)
    - 2.1 : Wheel speed (differential)
3. [Vision](#Partie-2-:-Vision)
    - 3.1 : Dynamic lighting adaptation
    - 3.2 : Aruko markers
        - 3.2.1 : Thymio and goal
    - 3.2 : Map resizing
4. [Kalmann](#Partie-3-:-Kalmann)
    - 4.1 : Theory
    - 4.2 : Implementation
5. [Global path planning](#Partie-4-:-Global-path-planning)
    - 5.1 : Dijkstra
6. [Local navigation](#Partie-5-:-Local-navigation)
    - 6.1 : Local obstacle detection
    - 6.2 : Potential fields
7. [Motion control](#Partie-5-:-Motion-control)
    - 7.1 : Differential drive
    - 7.2 : P-conntroler
8. [Demonstrations](#Partie-5-:-emonstrations)
9. [Conclusion](#IConclusion)
---



## Introduction
Dans cette section, nous introduisons le sujet et les objectifs de l'analyse.

---



## 1 : Setup

### 1.1 : Physical setup

```Explication des données utilisées.```

```python
# Exemple de code pour charger et afficher les données
import pandas as pd

data = pd.read_csv("data.csv")  # Remplacez par votre fichier
data.head()


### 1.2 : Code setup

## 2 : Initial tuning

### 2.1 Wheel speed (differential)

## 3: Vision

### 3.1 : Dynamic lighting adaptation

### 3.2 : Aruko markers

#### 3.2.1 : Thymio and goal

### 3.2 : Map resizing

## 4 : Kalmann

### 4.1 : Theory

### 4.2 : Implementation

## 5 : Global path planning
 


### 5.1 : Dijkstra

## 6 : Local navigation

### 6.1 : Local obstacle detection

### 6.2 : Potential fields

## 7 : Motion control

### 7.1 : Differential drive

### 7.2 : P-conntroler

## 8 : Demonstrations

---
## 9 : Conclusion
---

